In [21]:
%load_ext autoreload
%autoreload 2

import parse_data.prepare_data as prepare_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import globals
import data_strings
import data_extraction.extract_trial as extract_trial
import utils.cosine_similarity as cosine_similarity
import analysis.wall_visibility_and_choice as wall_visibility_and_choice
import data_extraction.get_indices as get_indices
import plotting.plot_probability_chose_wall as plot_probability_chose_wall
import plotting.flipped_rotated_trajectory_testing_functions as fr_funcs
import analysis.opponent_visibility as opponent_visibility

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Notebook to identify opponent visibility at trial start 

In [15]:
## Load data files

In [22]:
data_folder = data_strings.DATA_FOLDER
json_filenames_all_social = data_strings.JSON_FILENAMES_SOCIAL
json_filenames_all_solo = data_strings.JSON_FILENAMES_SOLO

In [23]:
json_filenames = json_filenames_all_social

In [ ]:
df, trial_list = prepare_data.prepare_data(data_folder, json_filenames, combine=False)

filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\first_experiments_2409\240913\2024-09-13_11-31-00_YansuJerrySocial.json


In [16]:
## calculate probabilities for specified conditions

In [11]:
# analysis variables
inferred_choice = True
current_fov = 110
chosen_wall_index=0
visible_wall_index=None
wall_sep = None

In [ ]:
(probabilities,
 trial_data) = opponent_visibility.probability_trial_outcome_given_conditions_all_sessions(trial_list, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            trial_type=globals.HIGH_LOW, debug=False)

wall_choice_probabilities = [probabilities['other_visible'].ravel(), probabilities['other_not_visible'].ravel()]

In [17]:
## Plot probabilities for this condition across visibility of Other

In [ ]:
plot_probability_chose_wall.boxplot_probability_choose_wall(wall_choice_probabilities, ['Other visible', 'Other not visible'], ylabel="P(Choose Low)")

### End of Demo

### Sandbox

In [ ]:
## Testing atomic function outputs for a single trial

In [ ]:
player_id = 0

In [21]:
# filter the trial list for trialtype and wallsep

# filter trial list for HighLow trialtype
trial_indices = get_indices.get_trials_trialtype(trial_list, trial_type=globals.HIGH_LOW)
trial_list_filtered = [trial_list[i] for i in trial_indices]

# filter trial list for wall separations if specified
if wall_sep:
    trial_indices = get_indices.get_trials_with_wall_sep(trial_list_filtered, wall_sep=None)
    trial_list_filtered = [trial_list_filtered[i] for i in trial_indices]


In [22]:
trial_list_one = [trial_list_filtered[0]]
sandbox_trial_list = trial_list_filtered

In [ ]:
head_angle_vector = opponent_visibility.get_player_headangle_vector_slice_onset(player_id, trial_list=sandbox_trial_list, trial_index=0)
head_angle_vector

In [ ]:
self_position, other_position = opponent_visibility.get_two_player_positions_slice_onset(player_id, trial_list=sandbox_trial_list, trial_index=0)
self_position, other_position

In [ ]:
orientation_angle_to_other_session = opponent_visibility.get_angle_of_opponent_from_player_session(player_id, sandbox_trial_list)
np.rad2deg(orientation_angle_to_other_session)

In [ ]:
other_visible_session = opponent_visibility.get_other_visible_session(orientation_angle_to_other_session, current_fov=110)
other_visible_session

Walking through the process for getting P(Choose High) for trials where the opponent is visible and High is first seen

In [ ]:
# filter Other initially visible
(trial_list_filtered_other_visible,
  other_visible_trial_indices) = opponent_visibility.filter_trials_other_visible(sandbox_trial_list, other_visible_session)
len(other_visible_trial_indices)

In [ ]:
# filter both walls initially visible
(trial_list_filtered_both_walls_visible,
  both_walls_visible_indices) = opponent_visibility.filter_trials_both_walls_initially_visible(trial_list_filtered_other_visible, player_id, current_fov)
both_walls_visible_indices

In [ ]:
# filter high wall initially visible
(trial_list_filtered_high_wall_initially_visible,
  high_wall_initially_visible_indices) = opponent_visibility.filter_trials_one_wall_initially_visible(trial_list_filtered_other_visible, player_id,
                                                                                   wall_index=0, current_fov=current_fov)
len(high_wall_initially_visible_indices)

In [ ]:
(trial_list_filtered_player_choice,
  player_chose_given_wall_indices) = opponent_visibility.filter_trials_player_chose_given_wall(trial_list_filtered_high_wall_initially_visible, player_id,
                                                                            inferred_choice=True, given_wall_index=0)
len(player_chose_given_wall_indices)

In [ ]:
# test the player choice function is working correctly by printing the player choice array
# and the high wall number for all trials of this session

player_choice = wall_visibility_and_choice.get_player_wall_choice(sandbox_trial_list, player_id,
                                                                    inferred_choice=False, debug=False)

high_wall = np.full(len(sandbox_trial_list), np.nan)
for i, trial in enumerate(sandbox_trial_list):
    high_wall[i] = get_indices.get_walls(trial)[0]

player_choice, high_wall

In [ ]:
probability_chose_wall = opponent_visibility.calculate_probability_choose_wall(trial_list_filtered_high_wall_initially_visible, trial_list_filtered_player_choice)
probability_chose_wall

In [20]:
## Testing for single trials

In [ ]:
# # Single trials
# P(H | H first visible; Other visible)
a = opponent_visibility.probability_chose_wall_conditioned_on_wall_and_other_visibility(sandbox_trial_list, other_visible_session,
                                                                    player_id=0, visible_wall_index=0,
                                                                    chosen_wall_index=0, inferred_choice=False,
                                                                    current_fov=current_fov,
                                                                    inverse_other_visible=False)
# P(H | H first visible; Other not visible)
b = opponent_visibility.probability_chose_wall_conditioned_on_wall_and_other_visibility(sandbox_trial_list, other_visible_session,
                                                                    player_id=0, visible_wall_index=0,
                                                                    chosen_wall_index=0, inferred_choice=False,
                                                                    current_fov=current_fov,
                                                                    inverse_other_visible=True)
wall_choice_probabilities = np.array([a,b])

In [ ]:
# Test plot for single trials
plot_probability_chose_wall.boxplot_probability_choose_wall(wall_choice_probabilities, ['Other visible', 'Other not visible'], ylabel="P(Choose High | first visible)")

In [ ]:
# plot trial start positions for self and other, combining both players in a session, 
# only for trials with Other visible and H visible at trial start
# TODO rewrite cleanly to avoid 
# TODO repeat for the list of trials where Other is NOT visible
# TODO see 250123 octagon meeting notes for how to polish these plots and best use them

chosen_walls_session = np.zeros(len(filtered_trail_list_a_list[0]))  # unused for this analysis

# trial lists
filtered_trail_list_a_list          # all trials with Other and High visible
filtered_trial_list_choice_a_list   # all trials with Other not viisble and High visible

# starting with player 0, the first index 
player_id = 0

# Run the first trial out of the loop to generate the plotted octagon axes
trial = filtered_trail_list_a_list[12][0]
(_, _, 
rotated_flipped_trial, _,
chosen_wall) = fr_funcs.get_trajectory_information_trial(chosen_walls_session, trial=trial, player_id=0)
ax = fr_funcs.plot_player_start_positions(rotated_flipped_trial, chosen_player=player_id, label=False, axes=None)

# now loop the rest of player_id 0 trials
for i, trial in enumerate(filtered_trail_list_a_list[12][1:]):
    (_, _, 
    rotated_flipped_trial, _,
    chosen_wall) = fr_funcs.get_trajectory_information_trial(chosen_walls_session, trial=trial, player_id=player_id)
    ax = fr_funcs.plot_player_start_positions(rotated_flipped_trial, chosen_player=player_id, label=False, axes=ax)
# now loop all player_id 1 trials
player_id = 1
for i, trial in enumerate(filtered_trail_list_a_list[13]):
    (_, _, 
    rotated_flipped_trial, _,
    chosen_wall) = fr_funcs.get_trajectory_information_trial(chosen_walls_session, trial=trial, player_id=player_id)
    ax = fr_funcs.plot_player_start_positions(rotated_flipped_trial, chosen_player=player_id, label=False, axes=ax)


In [ ]:
filtered_trail_list_a_list[17][0]['data.trialNum'].unique().item()